In [1]:
#Implement some functions

import io
import os
import unicodedata
import string
import glob

import torch
import random
import warnings
warnings.filterwarnings("ignore", message=".*NumPy.*")

# ALL_LETTERS: alphabet small + capital letters + " .,;'"
ALL_LETTERS = string.ascii_letters + " .,;'"
# N_LETTERS: the number of characters
N_LETTERS = len(ALL_LETTERS)

# Turn a Unicode string to plain ASCII
def unicode_to_ascii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
        and c in ALL_LETTERS
    )

def load_data():
    # Build the category_lines dictionary, a list of names per language
    # category_lines: all names in each language
    category_lines = {}
    # all_categories: all languages
    all_categories = []
    
    def find_files(path):
        return glob.glob(path)
    
    # Read a file and split into lines
    def read_lines(filename):
        lines = io.open(filename, encoding='utf-8').read().strip().split('\n')
        return [unicode_to_ascii(line) for line in lines]
        
    # Find the language and corresponding names
    for filename in find_files('Data/*.txt'):
        category = os.path.splitext(os.path.basename(filename))[0]
        all_categories.append(category)
        
        lines = read_lines(filename)
        category_lines[category] = lines
        
    return category_lines, all_categories

# Find letter index from all_letters, e.g. "a" = 0
def letter_to_index(letter):
    return ALL_LETTERS.find(letter)

# Turn a letter into a <1 x n_letters> Tensor
def letter_to_tensor(letter):
    tensor = torch.zeros(1, N_LETTERS)
    tensor[0][letter_to_index(letter)] = 1
    return tensor

# Turn a name into a <line_length x 1 x n_letters> (the one-hot vector)
def line_to_tensor(line):
    tensor = torch.zeros(len(line), 1, N_LETTERS)
    for i, letter in enumerate(line):
        tensor[i][0][letter_to_index(letter)] = 1
    return tensor

# Randomly select a training sample
def random_training_example(category_lines, all_categories):
    # Choose a random number
    def random_choice(a):
        random_idx = random.randint(0, len(a) - 1)
        return a[random_idx]
        
    # Find a random traning language and its names
    category = random_choice(all_categories)
    line = random_choice(category_lines[category])
    category_tensor = torch.tensor([all_categories.index(category)], dtype=torch.long)
    line_tensor = line_to_tensor(line)
    return category, line, category_tensor, line_tensor




A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.1 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/opt/anaconda3/envs/Pytorch/lib/python3.12/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/opt/anaconda3/envs/Pytorch/lib/python3.12/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/opt/anaconda3/envs/Pytorch/lib/python3.12/site-packages/ipykernel/kernelapp.py", line 701, in start
    self.io_loop.start()
  Fi

In [2]:
import torch 
import torch.nn as nn
import matplotlib.pyplot as plt

In [ ]:
# Define the RNN model 
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN,self).__init__()
        self.hidden_size = hidden_size
        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(input_size + hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)
    def forward(self,input_tensor,hidden_tensor):
        combined = torch.cat((input_tensor, hidden_tensor),1)
        hidden = self.i2h(combined)
        output = self.i2o(combined)
        output = self.softmax(output)
        return output, hidden
    def init_hidden(self):
        return torch.zeros(1,self.hidden_size)


category_lines, all_categories = load_data()
# n_categories: number of categories
n_categories = len(all_categories)

# n_hidden: number of hidden state
n_hidden = 128
rnn = RNN(N_LETTERS, n_hidden, n_categories)

# Predict category of a name
def category_from_output(output):
    category_idx = torch.argmax(output).item()
    return all_categories[category_idx]

# print(category_from_output(output))

# Train the RNN
criterion = nn.NLLLoss()
learning_rate = 0.005
optimizer = torch.optim.SGD(rnn.parameters(),lr=learning_rate)


def train(line_tensor, category_tensor):
    # Initialize hidden state
    hidden = rnn.init_hidden()
    # Update hidden state and produce output
    for i in range(line_tensor.size()[0]):
        output, hidden = rnn(line_tensor[i],hidden)
    # Loss calculation
    loss = criterion(output, category_tensor)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    return output, loss.item()

current_loss = 0
all_losses = []
plot_steps, print_steps = 1000, 5000
n_iters = 100000

# Record the loss function
for i in range(n_iters):
    category, line, category_tensor, line_tensor = random_training_example(category_lines, all_categories)
    output, loss = train(line_tensor, category_tensor)
    current_loss += loss
    if (i+1) % plot_steps == 0:
        all_losses.append(current_loss / plot_steps)
        current_loss = 0
    if (i+1) % print_steps == 0:
        guess = category_from_output(output)
        correct = "CORRECT" if guess == category else f"WRONG({category})"
        print(f"{i} {i/n_iters*100} {loss:.4f} {line}/{guess} {correct}")

# Plot the loss function
plt.figure()
plt.plot(all_losses)
plt.show()

# Predict the category of a new name  
def predict(input_line):
    print(f"\n>{input_line}")
    with torch.no_grad():
        line_tensor = line_to_tensor(input_line)
        hidden = rnn.init_hidden()
        for i in range(line_tensor.size()[0]):
            output, hidden = rnn(line_tensor[i],hidden)
    guess = category_from_output(output)
    print(guess)

# Get input from users
while True:
    sentence = input("Input:")
    if sentence == "quit":
        break
    predict(sentence)

4999 4.999 2.6919 Pinho/Italian WRONG(Portuguese)
9999 9.998999999999999 2.3187 Cockle/Irish WRONG(English)
14999 14.999 3.1181 Santiago/Japanese WRONG(Portuguese)
19999 19.999 2.3024 Borde/Portuguese WRONG(French)
24999 24.999 1.2113 Doan/Vietnamese CORRECT
29999 29.999 1.4461 Awad/Arabic CORRECT
34999 34.999 2.4315 Kaglantge/French WRONG(Greek)
39999 39.999 2.7600 Santiago/Japanese WRONG(Spanish)
44999 44.999 0.7638 Si/Korean CORRECT
49999 49.999 1.6139 Walker/Scottish CORRECT
54999 54.998999999999995 2.3764 Linville/Italian WRONG(French)
59999 59.999 1.7350 Theohari/Japanese WRONG(Greek)
64999 64.999 0.1530 Son/Korean CORRECT
69999 69.999 0.5018 Diep/Vietnamese CORRECT
74999 74.99900000000001 0.2892 Ta/Vietnamese CORRECT
